# wps_parameters

#### wps_parameters is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [ ]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools import copy_http_content

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [ ]:
# NBVAL_IGNORE_OUTPUT
osprey.parameters?

#### We can use the docstring to ensure we provide the appropriate parameters.

In [ ]:
# run parameters
params_output = osprey.parameters(
    params_config = "tests/data/configs/parameters_local.cfg"
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [ ]:
params_data = Dataset(make_tmp_copy(params_output))

# Metadata of the output of RVIC parameters module
params_meatadata = {
    "title": params_data.title,
    "year": params_data.history.split()[5],
    "month": params_data.history.split()[2],
    "day": params_data.history.split()[3],
    "RvicPourPointsFile": params_data.RvicPourPointsFile,
    "RvicUHFile": params_data.RvicUHFile,
    "RvicFdrFile": params_data.RvicFdrFile,
    "RvicDomainFile": params_data.RvicDomainFile,
}

In [ ]:
params_config = read_config("tests/data/configs/parameters_local.cfg")

# Expected output metadata determined from the input configuration file
expected_metadata["RvicPourPointsFile"] = params_config["POUR_POINTS"]["FILE_NAME"].split("/")[-1]
expected_metadata["RvicUHFile"] = params_config["UH_BOX"]["FILE_NAME"].split("/")[-1]
expected_metadata["RvicFdrFile"] = params_config["ROUTING"]["FILE_NAME"].split("/")[-1]

assert params_meatadata == expected_metadata